## What is this?
This tool is used to take the raw restaurant addresses scraped websites and run them through the Google Maps API to validate and format them in a standard way.
It also extracts the City, State, and Zip info.

In [12]:
import pandas as pd
from google.oauth2 import service_account
from google.maps import addressvalidation_v1
import config

# Fill this in with your Google Maps API credentials JSON path
google_api_credentials = service_account.Credentials.from_service_account_file(
    config.GOOGLE_MAPS_API_CREDENTIALS_JSON_PATH
)

async def get_validated_address(credentials: service_account.Credentials, address: str):
    client = addressvalidation_v1.AddressValidationAsyncClient(credentials=credentials)

    request = addressvalidation_v1.ValidateAddressRequest(
        address={
            'region_code': 'US',
            'address_lines': [address],
        },
    )

    return  await client.validate_address(request=request)


# Read dataframe from PKL file
df = pd.read_pickle('food_network_restaurants.pkl.zst', compression='zstd')

for index, row in df.iterrows():
    try:
        validation_response = await get_validated_address(google_api_credentials, row['address'])

        df.loc[index, 'validated_address'] = validation_response.result.address.formatted_address
        df.loc[index, 'city'] = validation_response.result.address.postal_address.locality
        df.loc[index, 'state'] = validation_response.result.address.postal_address.administrative_area
        df.loc[index, 'zip'] = validation_response.result.address.postal_address.postal_code

    except Exception as e:
        print('Error processing address', row['address'], e)

df.to_pickle('food_network_restaurants_with_validated_addresses.pkl.zst', compression='zstd')


Error processing address 818 Main St., Rapid City, SD 57701 500 Internal error encountered.
Error processing address  400 Address lines missing from request.
Error processing address  400 Address lines missing from request.
Error processing address 222 S Palm Canyon Dr                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           